#  Display options:

In [ ]:

#  References for this Notebook,
#
#     Working thru the examples on this page,
#        https://docs.k9h.dev/latest/recipes/udf-recipes.html/tree/master/lonestar/analytics/distributed/experimental
# 


In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


#  Setup stuff: Connectivity

In [ ]:

from katana import remote
#  from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


In [ ]:

display(my_graph.num_nodes())
display(my_graph.num_edges())


# Working with num_partitions()

In [ ]:
#  Even though this graph was made with 3 partitions, and even though certain operations
#  require at least 3 partitions else they fail, you may still see num_partitions = None
#  which can happen on really small graphs, and produce unexpected results below.

print(my_graph.num_partitions)


In [ ]:

#  my_graph.repartition(num_partitions = 1)
my_graph.repartition(num_partitions = 3)

print("--")


# UDFs, Example 01: Count City Nodes

In [18]:

#  All of these below are translated to the Airport graph
#

#  Not an exact copy of example 1, some things left off-
#
def my_func(i_graph, i_property):
    
   l_airport_seq = i_graph.nodes(labels = i_property)
      #
   print("Type: %s" % (str(type(l_airport_seq))))
   print("Value: %s" % (l_airport_seq))
   print("Loop thru the above:")
      #
   for l_each in l_airport_seq:
      print("   " + str(l_each))
        
   print("Airport Node Count", len(l_airport_seq))

   return

l_result = my_graph.run(lambda g: my_func(g, "Airport"))


#  Sample output,
#
#     Host 0 output:
#     Type: <class 'katana.distributed.EntitySequenceNode'>
#     Value: <katana.distributed.EntitySequenceNode object at 0x7f8da9fba730>
#     Loop thru the above:
#        <lnid 4294967294>
#        <lnid 0>
#        <lnid 1>
#     
#     Host 1 output:
#     Type: <class 'katana.distributed.EntitySequenceNode'>
#     Value: <katana.distributed.EntitySequenceNode object at 0x7f90c5ab63f0>
#     Loop thru the above:
#        <lnid 4294967293>
#        <lnid 4294967294>
#        <lnid 0>
#     
#     Host 2 output:
#     Type: <class 'katana.distributed.EntitySequenceNode'>
#     Value: <katana.distributed.EntitySequenceNode object at 0x7fe5882db7b0>
#     Loop thru the above:
#        <lnid 4294967293>
#        <lnid 4294967294>
#        <lnid 0>


          0/? [?op/s]


Host 0 output:
Type: <class 'katana.distributed.EntitySequenceNode'>
Value: <katana.distributed.EntitySequenceNode object at 0x7f8da9fbaf30>
Loop thru the above:
   <lnid 4294967294>
   <lnid 0>
   <lnid 1>
Airport Node Count 3

Host 1 output:
Type: <class 'katana.distributed.EntitySequenceNode'>
Value: <katana.distributed.EntitySequenceNode object at 0x7f90c5ab6df0>
Loop thru the above:
   <lnid 4294967293>
   <lnid 4294967294>
   <lnid 0>
Airport Node Count 3

Host 2 output:
Type: <class 'katana.distributed.EntitySequenceNode'>
Value: <katana.distributed.EntitySequenceNode object at 0x7fe5882e4230>
Loop thru the above:
   <lnid 4294967293>
   <lnid 4294967294>
   <lnid 0>
Airport Node Count 3


In [17]:

#  All of these below are translated to the Airport graph
#

#  An exact copy of example 1
#
def my_func(i_graph, i_property):
    
   l_airport_seq: Sequence[Node] = i_graph.nodes(labels = i_property)
      #
   print("Type: %s" % (str(type(l_airport_seq))))
   print("Value: %s" % (l_airport_seq))
   print("Loop thru the above:")
      #
   for l_each in l_airport_seq:
      print("   " + str(l_each))

   print("Airport Node Count", len(l_airport_seq))

   return

l_result = my_graph.run(lambda g: my_func(g, "Airport"))


#  Sample output,
#




    
    


          0/? [?op/s]


Host 0 output:
Type: <class 'katana.distributed.EntitySequenceNode'>
Value: <katana.distributed.EntitySequenceNode object at 0x7f8da9fba5f0>
Loop thru the above:
   <lnid 4294967294>
   <lnid 0>
   <lnid 1>

Host 1 output:
Type: <class 'katana.distributed.EntitySequenceNode'>
Value: <katana.distributed.EntitySequenceNode object at 0x7f90c5ab6870>
Loop thru the above:
   <lnid 4294967293>
   <lnid 4294967294>
   <lnid 0>

Host 2 output:
Type: <class 'katana.distributed.EntitySequenceNode'>
Value: <katana.distributed.EntitySequenceNode object at 0x7fe5882dbcb0>
Loop thru the above:
   <lnid 4294967293>
   <lnid 4294967294>
   <lnid 0>
